# Clase Práctica 01

# Neural Network - Redes Neuronales Artificiales

# Primera Red Neuronal con Keras



# Introducción

Este tutorial muestra el flujo de trabajo básico de usar Keras con un modelo lineal simple, se va a explicar y desarrollar un código básico con Tensorflow y Keras.

Los pasos a seguir para el desarrollo del código son los siguientes: 

1. Cargar los datos.
2. Definir nuestro modelo.
3. Compilar nuestro modelo.
4. Ajustar nuestro modelo.
5. Evaluar nuestro modelo.
6. Unir todo el proceso.



# Base de datos: Pima Indians

En este tutorial vamos a utilizar la base de datos de diabetes Pima Indians. La base de datos describe los datos de los registros médicos de los pacientes que tuvieron un inicio de diabetes dentro de los cinco años. Corresponde a un problema de clasificación binaria (aparición de diabetes como 1 o no aparición como 0). Las variables de entrada que describen a cada paciente son numéricas y tienen escalas variables. La base de datos posee 768 instancias. 

A continuación se enumeran los ocho atributos (descritos en inglés) para el conjunto de datos:

1. Number of times pregnant.
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skin fold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.



# Cargar librerías y datos  

Se utiliza una semilla (seed) para hacer los experimentos de Deep Learning repetibles. Esto es útil si necesita demostrar un resultado, comparar algoritmos usando la misma fuente de aleatoriedad o depurar una parte de su código. Puede inicializar el generador de números aleatorios con cualquier semilla que desee, por ejemplo: 7 como el que usamos en el ejemplo. 

In [152]:
# librerías
import torch
from torch import nn
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sequential
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.nn import functional as F
import numpy

# seed fija para experimentos
seed = 7
numpy.random.seed(seed)

Ahora podemos cargar nuestra base de datos. Se puede cargar el archivo directamente usando la función NumPy loadtxt(). Hay ocho variables de entrada y una variable de salida (la última columna contiene las etiquetas). Una vez cargado, podemos dividir el conjunto de datos en variables de entrada (X) y la variable de clase de salida (Y).

In [153]:
# cargar los datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# separamos en variables de entrada (X) y salidas (Y) 
inputs = torch.from_numpy(dataset[:,0:8])
X = DataLoader(inputs, batch_size=10, shuffle=True)
Y = torch.reshape(torch.from_numpy(dataset[:,8]), (-1,1))


print("Entradas:")
print(X)
print("Salidas:")
print(Y)
print("Observe que hay 768 ejemplos y 8 atributos:")
print(len(X))
print("Observe que hay 768 etiquetas")
print((Y.shape))

Entradas:
Salidas:
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
 

# Definir el modelo de red neuronal

Los modelos en Keras se definen como una secuencia de capas. Por lo tanto, vamos a crear un modelo secuencial y agregamos capas secuencialmente para obtener nuestra topología de red. Debemos asegurarnos que la capa de entrada que hemos definimos tenga el número correcto de entradas. Esto se puede especificar al crear la primera capa con el argumento llamado "dimensión de entrada" y establecerlo en 8 para las 8 variables de entrada.

¿Cómo sabemos el número de capas a usar y sus tipos? Esta es una pregunta muy difícil en Deep Learning. Podemos utilizar heurísticas y, a menudo, la mejor estructura de red se encuentra a través de un proceso de experimentación de prueba y error. En general, necesita una red lo suficientemente grande como para capturar la estructura del problema, si es que eso ayuda. En este ejemplo, utilizaremos una estructura de red totalmente conectada (fully connected) con tres capas.

Las capas totalmente conectadas (fully connected) se definen utilizando la clase "Dense". Podemos especificar el número de neuronas en la capa en el primer argumento; el método de inicialización de la red en el segundo argumento como init; y además, se puede especificar la función de activación utilizando el tercer argumento (función de activación). Para nuestro ejemplo, vamos a inicializar los pesos de la red de manera aleatoria, generada a partir de una distribución uniforme. Usaremos en este caso, valores entre 0 y 0.05, porque esa es la inicialización de pesos uniformes que trae de forma predeterminada Keras. Otra alternativa tradicional sería normal para los pequeños números aleatorios generados a partir de una distribución gaussiana.

Usaremos la función de activación de rectificación (ReLU) en las dos primeras capas y la función de activación sigmoide en la capa de salida. Tambien es posible usar las funciones de activación sigmoide y tanh para todas las capas. Actualente, se observa un mejor rendimiento (empírico) utilizando la función de activación de rectificación ReLU. Utilizamos una función de activación sigmoidea en la capa de salida para garantizar que nuestra salida de red esté entre 0 y 1, y sea fácil de asignar a una probabilidad. La primera capa oculta tiene 12 neuronas y espera 8 variables de entrada. La segunda capa oculta tiene 8 neuronas y solo la capa de salida tiene 1 neurona para predecir la clase (aparición de diabetes o no).

<img src="./images_tutoriales/Imagen1.png">


In [20]:
help(Linear)

Help on class Linear in module torch.nn.modules.linear:

class Linear(torch.nn.modules.module.Module)
 |  Linear(in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies a linear transformation to the incoming data: :math:`y = xA^T + b`
 |  
 |  This module supports :ref:`TensorFloat32<tf32_on_ampere>`.
 |  
 |  Args:
 |      in_features: size of each input sample
 |      out_features: size of each output sample
 |      bias: If set to ``False``, the layer will not learn an additive bias.
 |          Default: ``True``
 |  
 |  Shape:
 |      - Input: :math:`(*, H_{in})` where :math:`*` means any number of
 |        dimensions including none and :math:`H_{in} = \text{in\_features}`.
 |      - Output: :math:`(*, H_{out})` where all but the last dimension
 |        are the same shape as the input and :math:`H_{out} = \text{out\_features}`.
 |  
 |  Attributes:
 |      weight: the learnable weights of the module of shape
 |          :math:`(

In [5]:
from torch.nn import init 
model = Sequential()

In [6]:
model_naive = Sequential()
#add model layers
model.add_module("input", Linear(8, 12))
init.uniform_(model.input.weight)
init.uniform_(model.input.bias)
#----
model.add_module('relu1', nn.ReLU())
#----
model.add_module('linear', nn.Linear(12,8))
init.uniform_(model.linear.weight)
init.uniform_(model.linear.bias)
#----
model.add_module('relu2', nn.ReLU())
#----
model.add_module('output', nn.Linear(8, 1))
init.uniform_(model.output.weight)
init.uniform_(model.output.bias)
#----
model.add_module('sigmoid', nn.Sigmoid())
model

Sequential(
  (input): Linear(in_features=8, out_features=12, bias=True)
  (relu1): ReLU()
  (linear): Linear(in_features=12, out_features=8, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Esto puede ser un poco engorroso y para solucionarlo podemos inicializar los pesos de la siguiente forma: 
Extraido de: 

* https://pytorch.org/docs/stable/notes/modules.html?highlight=accuracy

In [79]:
#otra forma de defnir el modelo
model = Sequential(OrderedDict([
          ("input", Linear(8, 12)),
          ('relu1', nn.ReLU()),
          ('linear', nn.Linear(12,8)),
          ('relu2', nn.ReLU()),
          ('output', nn.Linear(8, 1)),

        ]))

model

Sequential(
  (input): Linear(in_features=8, out_features=12, bias=True)
  (relu1): ReLU()
  (linear): Linear(in_features=12, out_features=8, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [80]:
#linear puede ser cambiado por cualquier nombre de las otras capas 
pesos = model.linear.weight
bias = model.linear.bias

print(pesos.shape,bias.shape)
print(pesos,bias)

torch.Size([8, 12]) torch.Size([8])
Parameter containing:
tensor([[ 0.0429,  0.2882,  0.2692, -0.1615,  0.1010, -0.2043,  0.0157, -0.0557,
          0.2157,  0.1817,  0.1335, -0.1017],
        [-0.1936, -0.2346,  0.1623, -0.1967,  0.2627,  0.0396, -0.2027,  0.2675,
          0.1759, -0.0266,  0.1427,  0.0384],
        [ 0.1669,  0.0906,  0.2289,  0.0847, -0.1083, -0.1504,  0.0339,  0.1638,
          0.1590,  0.2767, -0.2238, -0.2117],
        [ 0.0070, -0.1554, -0.0123, -0.1936,  0.2054,  0.1190,  0.0012, -0.2223,
          0.1425, -0.2854, -0.1095,  0.0374],
        [ 0.1161, -0.2527, -0.0061, -0.2737,  0.0707, -0.2054,  0.2824,  0.0731,
         -0.2023,  0.1249,  0.0180, -0.1334],
        [-0.1381, -0.1896, -0.1990,  0.2315, -0.1568, -0.1135,  0.0724, -0.0216,
          0.0465, -0.0266, -0.1130, -0.1234],
        [ 0.2329, -0.2480, -0.1953, -0.2039, -0.1304, -0.2872,  0.0079,  0.1870,
         -0.2017, -0.1302,  0.0950, -0.2063],
        [-0.1886, -0.0412,  0.1489,  0.0704, -0.0595,

In [81]:
# Define a function to initialize Linear weights.
# Note that no_grad() is used here to avoid tracking this computation in the autograd graph
@torch.no_grad()
def init_weights(m):
  if isinstance(m, nn.Linear):
    nn.init.uniform_(m.weight)
    m.bias.fill_(0.0)

# Apply the function recursively on the module and its submodules.
model.apply(init_weights)

Sequential(
  (input): Linear(in_features=8, out_features=12, bias=True)
  (relu1): ReLU()
  (linear): Linear(in_features=12, out_features=8, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [82]:
#linear puede ser cambiado por cualquier nombre de las otras capas 
pesos = model.linear.weight
bias = model.linear.bias

print(pesos.shape,bias.shape)
print(pesos,bias)

torch.Size([8, 12]) torch.Size([8])
Parameter containing:
tensor([[0.7202, 0.9858, 0.3556, 0.6012, 0.1388, 0.1474, 0.0897, 0.1628, 0.0165,
         0.1500, 0.2205, 0.8756],
        [0.6651, 0.0448, 0.1978, 0.2625, 0.1477, 0.2835, 0.9380, 0.2549, 0.9815,
         0.9040, 0.4819, 0.3126],
        [0.0351, 0.1727, 0.0248, 0.4598, 0.6647, 0.9064, 0.5310, 0.1313, 0.4706,
         0.4232, 0.6050, 0.6337],
        [0.3525, 0.2669, 0.4928, 0.3760, 0.2052, 0.7902, 0.1324, 0.8206, 0.2773,
         0.2117, 0.1212, 0.2998],
        [0.1525, 0.9597, 0.6333, 0.0771, 0.7537, 0.7548, 0.6514, 0.2258, 0.8429,
         0.6457, 0.9362, 0.6753],
        [0.0868, 0.1705, 0.9799, 0.6545, 0.5480, 0.1481, 0.7384, 0.9542, 0.8401,
         0.6912, 0.4076, 0.7414],
        [0.2437, 0.5772, 0.7012, 0.6097, 0.6575, 0.6279, 0.9851, 0.3680, 0.1358,
         0.5426, 0.8795, 0.0054],
        [0.9236, 0.1757, 0.5425, 0.4305, 0.0136, 0.2243, 0.1128, 0.8333, 0.5360,
         0.2960, 0.4217, 0.2602]], requires_grad=True) P

In [128]:
#if result is more than 0.5 then it is 1 else 0
filt =lambda x:1 if x >0.5 else 0
 

(tensor([0.1187, 1.5644]), tensor([0.0000, 1.5644]))

In [142]:
#estilo pytorch
# Las redes (modelos en general) deben heredar desde torch.nn.Module
# modelo secuencial
class FFNN(Module):
  
  #definición 
  def __init__(self, d0=8, d1=12):
    super(FFNN, self).__init__()
    
    # Usa 'capas lineales' en vez de parámetros explícitos
    # Los parámetros se incializan automáticamente y se agregan
    # a los parámetros de la red
    self.fc_1 = Linear(d0,d1)#8 entradas y 12 neuronas en capa oculta
    self.fc_2 = Linear(d1,d0) # 8 neuronas en capa oculta 
    self.fc_out = Linear(d0,2)# 1 neurona en la salida
    self.relu = nn.ReLU()
    #self.sigmoid = nn.Sigmoid()
    
  def forward(self, x):
    h1 = self.relu(self.fc_1(x))
    h2 = self.relu(self.fc_2(h1))
    #y_pred = self.sigmoid(self.fc_out(h2))
    y_pred = self.fc_out(h2) 
    return y_pred

# Compilar el modelo de red neuronal

Ahora que el modelo está definido, podemos compilarlo. La compilación del modelo utiliza las bibliotecas numéricas eficientes llamadas backend, como TensorFlow o Theano. El backend elige automáticamente la mejor forma de representar la red para entrenarse y hacer predicciones para ejecutar en su hardware. Al compilar, debemos especificar algunas propiedades adicionales requeridas al entrenar la red. Recuerde que entrenar una red significa encontrar el mejor conjunto de pesos y sesgos para hacer predicciones para este problema.

Debemos especificar la función de pérdida (loss) que se usará para evaluar un conjunto de entrenamiento; tambien se debe especificar el optimizador para buscar los pesos óptimos de la red; y cualquier métrica que nos permita evaluar el desempeño  durante el entrenamiento. En este caso utilizaremos la pérdida logarítmica (logarithmic loss), que para un problema de **clasificación binaria** se define en Keras como **crosentropia binaria (binary_crossentropy)**. También utilizaremos el algoritmo de **descenso de gradiente** eficiente llamado **adam**. Finalmente, debido a que es un problema de clasificación, la métrica que vamos a recopilar e informar es la **precisión (accuracy)**.

In [146]:
# Establecemos loss, optimizador y metrica de evaluación
model= FFNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())


# Ajustar el modelo de red neuronal (Fit)

Hemos definido nuestro modelo y lo hemos compilado listo para poder realizar un cálculo eficiente. Ahora es el momento de ejecutar el modelo en algunos datos. Podemos entrenar nuestro modelo con nuestros datos cargados llamando a la función de ajuste **fit ()** en el modelo.

El proceso de entrenamiento se ejecutará para un número fijo de iteraciones a través del conjunto de datos llamado **épocas (epochs)**, que debemos especificar usando el argumento nb_epoch. También podemos establecer el número de instancias que se evalúan antes de que se realice una actualización de peso en la red, denominada **tamaño de lote (batch_size)** que es establecido con el argumento de **batch_size**. Para este problema ejecutaremos un pequeño número de épocas (150) y usaremos un tamaño de lote relativamente pequeño de 10. Una vez más, estos pueden ser seleccionados experimentalmente por prueba y error.

In [160]:

epochs=1000
# Train the network

model.train()
for epoch in range(epochs):

    iteration_loss = 0.
    iteration_accuracy = 0.
    for i, batch_data in enumerate(X):
        batch_size=len(batch_data)

        # Get the inputs
        inpu = batch_data.float()
        labels = Y[i*batch_size:(i+1)*batch_size].float()
        # Forward pass
        outputs = model.forward(inpu)

        # Compute loss
        batch_loss = criterion(outputs, labels)
        #Backward and optimize
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        # Compute statistics
        iteration_loss += batch_loss
        #iteration_accuracy += accuracy(outputs, labels)
        if(i % 10 == 0):
            print(f'Train: epoch: {epoch} -',
                  f'loss: {iteration_loss/(i+1):.5f}; ')

    
with torch.no_grad(): 
    model.eval() 
    inpu, outputs = inputs.float(), Y.float()
    predicted_outputs = model(inpu) 







Train: epoch: 0 - loss: 0.80285; 
Train: epoch: 0 - loss: 0.65043; 
Train: epoch: 0 - loss: 0.66080; 
Train: epoch: 0 - loss: 0.67045; 
Train: epoch: 0 - loss: 0.66934; 
Train: epoch: 0 - loss: 0.65423; 
Train: epoch: 0 - loss: 0.64505; 
Train: epoch: 0 - loss: 0.64539; 
Train: epoch: 1 - loss: 0.80285; 
Train: epoch: 1 - loss: 0.65043; 
Train: epoch: 1 - loss: 0.66080; 
Train: epoch: 1 - loss: 0.67045; 
Train: epoch: 1 - loss: 0.66934; 
Train: epoch: 1 - loss: 0.65423; 
Train: epoch: 1 - loss: 0.64505; 
Train: epoch: 1 - loss: 0.64539; 
Train: epoch: 2 - loss: 0.80285; 
Train: epoch: 2 - loss: 0.65043; 
Train: epoch: 2 - loss: 0.66080; 
Train: epoch: 2 - loss: 0.67045; 
Train: epoch: 2 - loss: 0.66934; 
Train: epoch: 2 - loss: 0.65423; 
Train: epoch: 2 - loss: 0.64505; 
Train: epoch: 2 - loss: 0.64539; 
Train: epoch: 3 - loss: 0.80285; 
Train: epoch: 3 - loss: 0.65043; 
Train: epoch: 3 - loss: 0.66080; 
Train: epoch: 3 - loss: 0.67045; 
Train: epoch: 3 - loss: 0.66934; 
Train: epoch: 

KeyboardInterrupt: 

In [157]:
outputs

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [120]:
# Ajustar el modelo
epochs=150
batch_size=10
train_loader=torch.tensor(X)
target = Y
data = train_loader
generator = torch.utils.data.DataLoader(
        data, batch_size=10, shuffle=True
    )
# Train the network
model.train()
train_loss, train_acc = 0.0, 0.0
for epoch in range(epochs):
  for i in range(0, len(data), batch_size):
    # Forward pass: compute predicted y by passing x to the model
    x = data[i:i+batch_size]
    y = target[i:i+batch_size]
    y_pred = model(x)

    
    # Compute and print loss
    batch_loss  = criterion(y_pred, y)
    print(f'Epoch {epoch+1}/{epochs} loss: {batch_loss.item():.4f}')
    
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    train_loss += batch_loss.item()
    train_acc += torch.sum(y_pred == y).item()

print(train_loss / len(data))
print(train_acc / len(data))

RuntimeError: Could not infer dtype of DataLoader

In [9]:
# Ajustar el modelo
epochs=150
batch_size=10
train_loader=X
target = Y

# Train the network
data = train_loader
for epoch in range(epochs):
    for batch_data  in enumerate(data):
        # Obtaining the cuda parameters

        target = target
        
        # Reshaping to suit our model
        data = data.reshape(data.shape[0], -1)
       
        # Forward propagation
        score = model(data)
        loss = criterion(score, target)
       
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch 1/150
77/77 [==============================] - 5s 8ms/step - loss: 0.6797 - accuracy: 0.6432
Epoch 2/150
77/77 [==============================] - 0s 6ms/step - loss: 0.6631 - accuracy: 0.6510
Epoch 3/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6589 - accuracy: 0.6497
Epoch 4/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6521 - accuracy: 0.6497
Epoch 5/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6448 - accuracy: 0.6549
Epoch 6/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6315 - accuracy: 0.6693
Epoch 7/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6226 - accuracy: 0.6602
Epoch 8/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6079 - accuracy: 0.6771
Epoch 9/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5985 - accuracy: 0.6940
Epoch 10/150
77/77 [==============================] - 0s 4ms/step - loss: 0.5951 - accuracy: 0.6940
Epoch 11/

# Evaluar el modelo de red neuronal 

Hemos entrenado nuestra red neuronal en **todo** el conjunto de datos y podemos evaluar el rendimiento de la red con el mismo conjunto de datos. Esto no se suele hacer, sino que esto solo **nos dará una idea de qué tan bien hemos modelado el conjunto de datos (por ejemplo, la precisión del entrenamiento), pero no tenemos idea de qué tan bien podría funcionar el algoritmo con los nuevos datos**. Lo hemos hecho para simplificar, pero lo ideal es que pueda separar sus datos en conjuntos de datos de entrenamiento y test, uno para el entrenamiento y otro para la evaluación de su modelo.

Puede evaluar su modelo en sus datos de entrenamiento utilizando la función evaluate() de su modelo y pasarle la misma entrada y salida utilizada para entrenar el modelo. Esto generará una predicción para cada par de entrada y salida, y recopilará el rendimiento de la red, incluida la pérdida promedio y cualquier métrica que haya configurado, como la precisión.

In [11]:
# evaluar el modelo
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

24/24 [==============================] - 0s 3ms/step - loss: 0.4304 - accuracy: 0.7865
accuracy: 78.65%


# Unir todo el proceso

Acaba de ver cómo puede crear fácilmente su primer modelo de red neuronal en Keras. Vamos a unirlo todo en un ejemplo de código completo.

In [163]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajustar
model.fit(X, Y, epochs=150, batch_size=10)
# evaluar
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
77/77 [==============================] - 8s 6ms/step - loss: 0.6855 - accuracy: 0.5951
Epoch 2/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6663 - accuracy: 0.6510
Epoch 3/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6531 - accuracy: 0.6523
Epoch 4/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6403 - accuracy: 0.6576
Epoch 5/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6248 - accuracy: 0.6562
Epoch 6/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6114 - accuracy: 0.6823
Epoch 7/150
77/77 [==============================] - 0s 4ms/step - loss: 0.5980 - accuracy: 0.6797
Epoch 8/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6031 - accuracy: 0.6745
Epoch 9/150
77/77 [==============================] - 0s 4ms/step - loss: 0.5905 - accuracy: 0.6914
Epoch 10/150
77/77 [==============================] - 0s 4ms/step - loss: 0.5936 - accuracy: 0.6953
Epoch 11/